In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# ------------------- Load Data -------------------
df = pd.read_csv("cleaned_hotstar_data.csv")

# Convert release_year to numeric
df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce')

# Drop missing years
df = df.dropna(subset=['release_year'])

# Count number of releases per year
release_counts = df['release_year'].value_counts().sort_index()
data = release_counts.values.reshape(-1, 1)
years = release_counts.index.tolist()

# ------------------- Normalize -------------------
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# ------------------- Create Sequences -------------------
def create_sequences(data, seq_len=3):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len])
    return np.array(X), np.array(y)

seq_len = 3
X, y = create_sequences(scaled_data, seq_len)

# ------------------- Build LSTM Model -------------------
model = Sequential([
    LSTM(50, activation='relu', input_shape=(X.shape[1], X.shape[2])),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=100, verbose=0)

# ------------------- Predict Next 5 Years -------------------
last_seq = scaled_data[-seq_len:]
predictions = []

for _ in range(5):
    pred = model.predict(last_seq.reshape(1, seq_len, 1), verbose=0)
    predictions.append(pred[0][0])
    last_seq = np.append(last_seq[1:], [[pred]], axis=0)

# Inverse scale predictions
predicted_counts = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()

# ------------------- Plot Results -------------------
plt.figure(figsize=(10, 5))
plt.plot(years, data, label="Original")
plt.plot(range(years[-1]+1, years[-1]+6), predicted_counts, label="LSTM Forecast", linestyle='--', marker='o')
plt.xlabel("Year")
plt.ylabel("Number of Releases")
plt.title("JioHotstar Content Forecast using LSTM")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/3c/e3/e868f1d5951047f950d2ba1e04a765a3328a51f06996b67976d6102f8227/tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/f6/d4/349f7f4bd5ea92dab34f5bb0fe31775ef6c311427a14d5a5b31ecb442341/absl_py-2.2.2-py3-none-any.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Obtaining dependency information for flatbuffers>=24.3.25 from https://files.pythonhosted.org/packages/b8/25/155f9f080d5e4bc0082edfda032ea2bc2b8fab3f4d25d46c1e9dd22a1a89/flatbuffers-25.2.10-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gast!=0.5.0,!=0.5.1,!=0.5.2,>=0

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\smrut\\anaconda3\\Lib\\site-packages\\~umpy\\core\\_multiarray_tests.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.

